# Terra Signal Hackathon
This notebook is provided as a starting point. Feel free to use it, discard it, modify it, or pretend it doesn't exist.

In [0]:
%pip install pandas

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file using pandas
file_path = "./history.csv"
df = pd.read_csv(file_path)
df.head().transpose()

In [0]:
df.info()

In [0]:
for col in df.columns:
    print(f"\n--- {col} ---")
    print(df[col].value_counts())


Limpeza dos dados

In [0]:
# removendo coluna irrelevante
df_clean = df.copy()
df_clean = df_clean.drop(columns=["customerID"])

# limpando a coluna 'tenure'

# substituindo 'unknown' por NaN e convertendo para numérico
df_clean["tenure"] = df_clean["tenure"].replace("unknown", pd.NA)
df_clean["tenure"] = pd.to_numeric(df_clean["tenure"], errors="coerce")

# substituindo valores 0 por NaN
df_clean.loc[df_clean["tenure"] == 0, "tenure"] = pd.NA

# preenchendo NaN com a mediana e convertendo para inteiro
df_clean["tenure"] = df_clean["tenure"].fillna(df_clean["tenure"].median())
df_clean["tenure"] = df_clean["tenure"].astype(int)

# normalizando phone service
df_clean["PhoneService"] = (
    df_clean["PhoneService"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"yes": 1, "no": 0})
    .astype(int)  # <- evitar FutureWarning
)

# normalizando multiple lines
df_clean["MultipleLines"] = (
    df_clean["MultipleLines"]
    .replace({"No phone service": "No"})
    .map({"Yes": 1, "No": 0})
    .astype(int)
)

# normalizando colunas de internet
internet_cols = [
    "OnlineSecurity", "OnlineBackup", "DeviceProtection",
    "TechSupport", "StreamingTV", "StreamingMovies"
]

for col in internet_cols:
    df_clean[col] = (
        df_clean[col]
        .replace({"No internet service": "No"})
        .map({"Yes": 1, "No": 0})
        .astype(int)
    )

# normalizando colunas binárias
for col in ["Partner", "Dependents", "PaperlessBilling"]:
    df_clean[col] = df_clean[col].map({"Yes": 1, "No": 0}).astype(int)

# convertendo total charges para numérico e tratando NaN
df_clean["TotalCharges"] = pd.to_numeric(df_clean["TotalCharges"], errors="coerce")
df_clean["TotalCharges"] = df_clean["TotalCharges"].fillna(df_clean["TotalCharges"].median())

# limpando coluna de feedback do cliente
df_clean["CustomerFeedback"] = df_clean["CustomerFeedback"].fillna("").astype(str)
df_clean["CustomerFeedback_clean"] = (
    df_clean["CustomerFeedback"]
    .str.lower()
    .str.replace("[^a-zA-Z0-9 ]", "", regex=True)
)

# tratando categóricas com one-hot encoding
cat_cols = [
    "gender", "InternetService", "Contract", "PaymentMethod"
]

df_clean = pd.get_dummies(df_clean, columns=cat_cols, drop_first=True)

# convertendo target para binário
df_clean["Churn"] = df_clean["Churn"].map({"Yes": 1, "No": 0}).astype(int)


In [0]:
df_clean.info()


In [0]:
df_clean.head()

In [0]:
df_clean.to_csv("history_clean.csv", index=False)

In [0]:
import datetime

def prediction_function(input_df):
    '''
    An example model function, that just predicts randomly whether a customer will churn.
    TODO: Make a better model.
    '''
    X = input_df[['customerID']].copy()
    X['prediction'] = np.random.uniform(size=len(X)) >= 0.5
    X['prediction'] = X['prediction'].map({True: 'Yes', False: 'No'})
    return X

test_df = pd.read_csv('inference.csv')
prediction = prediction_function(test_df)
print(prediction.head().transpose())
# Use this code to save the prediction to a csv file for submission:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
prediction.to_csv(f'prediction_<MY_GROUP_NAME>_{timestamp}.csv')

Análise Exploratória

In [0]:
plt.figure(figsize=(6,4))
sns.countplot(
    data=df_clean,
    x="Churn",
    hue="Churn",
    palette="Set2",
    legend=False
)
plt.title("Quantidade de Clientes: Churn vs Não Churn")
plt.xticks([0,1], ["Não churn", "Churn"])
plt.ylabel("Quantidade")
plt.xlabel("")
plt.show()


In [0]:
plt.figure(figsize=(10,5))
sns.histplot(
    data=df_clean,
    x="tenure",
    hue="Churn",
    multiple="stack",
    bins=40,
    palette="Set2"
)
plt.title("Distribuição de Tenure por Churn")
plt.show()


In [0]:
plt.figure(figsize=(8,5))
sns.boxplot(
    data=df_clean,
    x="Churn",
    y="MonthlyCharges",
    hue="Churn",
    palette="Set2",
    legend=False   # evita legenda duplicada
)
plt.title("MonthlyCharges por Churn")
plt.xticks([0,1], ["Não churn", "Churn"])
plt.show()


In [0]:
plt.figure(figsize=(8,5))
sns.violinplot(data=df_clean, x="Churn", y="TotalCharges", hue="Churn", palette="Set2", legend=False)
plt.title("Total Charges por Churn")
plt.show()


In [0]:
# crosstab normalizada por linha (proporção)
ct = pd.crosstab(
    df_clean["InternetService_Fiber optic"],
    df_clean["Churn"],
    normalize='index'
)

# garantir ordem correta das colunas
ct.columns = ["No", "Yes"]

# definir cores (No = cinza, Yes = verde)
colors = ["#C0C0C0", "#2ECC71"]

# plotar com as cores desejadas
ct.plot(
    kind="bar",
    stacked=True,
    figsize=(6,4),
    color=colors
)

plt.title("Composição: churn vs não churn — Fiber Optic (0 = não, 1 = sim)")
plt.ylabel("Proporção")
plt.xlabel("Fiber Optic")
plt.legend(title="Churn")
plt.ylim(0,1)
plt.show()


In [0]:
ct2 = pd.crosstab(
    df_clean["InternetService_No"],
    df_clean["Churn"],
    normalize='index'
)

# garantir ordem correta das colunas
ct2.columns = ["No", "Yes"]

# definir cores (No = cinza, Yes = verde)
colors = ["#C0C0C0", "#2ECC71"]

# plotar com as cores desejadas
ct2.plot(
    kind="bar",
    stacked=True,
    figsize=(6,4),
    color=colors
)

plt.title("Composição: churn vs não churn — No Internet (0 = não, 1 = sim)")
plt.ylabel("Proporção")
plt.xlabel("No Internet")
plt.ylim(0,1)
plt.legend(title="Churn")
plt.show()


In [0]:
plt.figure(figsize=(14,10))
corr = df_clean.corr(numeric_only=True)
sns.heatmap(corr, annot=False, cmap="coolwarm", linewidths=.5)
plt.title("Matriz de Correlação - Variáveis Numéricas")
plt.show()


In [0]:
plt.figure(figsize=(8,5))
sns.countplot(data=df_clean, x="Contract_Two year", hue="Churn")
plt.title("Churn por tipo de contrato (Ex.: Two Year)")
plt.show()


In [0]:
plt.figure(figsize=(8,5))
sns.countplot(data=df_clean, x="Contract_One year", hue="Churn")
plt.title("Churn por tipo de contrato (One Year)")
plt.show()


In [0]:
sns.pairplot(df_clean[["tenure","MonthlyCharges","TotalCharges","Churn"]], hue="Churn")
plt.show()


In [0]:
df_clean_spark = spark.createDataFrame(df_clean)

# mapeamento explícito dos nomes inválidos -> válidos
rename_map = {
    "InternetService_Fiber optic": "InternetService_Fiber_optic",
    "Contract_One year": "Contract_One_year",
    "Contract_Two year": "Contract_Two_year",
    "PaymentMethod_Credit card (automatic)": "PaymentMethod_Credit_card_automatic",
    "PaymentMethod_Electronic check": "PaymentMethod_Electronic_check",
    "PaymentMethod_Mailed check": "PaymentMethod_Mailed_check",
}

from functools import reduce

df_renamed = df_clean_spark
for old, new in rename_map.items():
    if old in df_renamed.columns:
        df_renamed = df_renamed.withColumnRenamed(old, new)

df_renamed.printSchema()

In [0]:
# vou assumir que o DataFrame com esse schema se chama df_renamed
df_renamed.printSchema()  # só pra garantir que é esse mesmo

# 1) criar o schema (database) dentro do catálogo workspace
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.churn")

# 2) salvar como tabela gerenciada Delta
tabela = "workspace.churn.history_genie"

(df_renamed.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable(tabela)
)

print("Tabela criada:", tabela)

In [0]:
# tabela que você criou pro Genie
df = (
    spark.table("workspace.churn.history_genie")
          .select("Churn", "CustomerFeedback_clean")
)

pdf = df.toPandas()
pdf = pdf.dropna(subset=["CustomerFeedback_clean"])
pdf.head()

In [0]:
pdf["tokens"] = pdf["CustomerFeedback_clean"].str.split()

# 0 = não churn (positivo), 1 = churn (negativo)
pdf_pos = pdf[pdf["Churn"] == 0]
pdf_neg = pdf[pdf["Churn"] == 1]

In [0]:
from collections import Counter

pos_counts = Counter(
    w
    for tokens in pdf_pos["tokens"]
    for w in tokens
)

neg_counts = Counter(
    w
    for tokens in pdf_neg["tokens"]
    for w in tokens
)

len(pos_counts), len(neg_counts)

In [0]:
import numpy as np
import pandas as pd

all_words = set(pos_counts) | set(neg_counts)

total_pos = sum(pos_counts.values())
total_neg = sum(neg_counts.values())
V = len(all_words)  # vocabulário para suavização

rows = []
for w in all_words:
    pos = pos_counts.get(w, 0)
    neg = neg_counts.get(w, 0)
    total = pos + neg

    # filtra palavras muito raras (ajuste o limiar)
    if total < 5:
        continue

    # probabilidades suavizadas
    p_pos = (pos + 1) / (total_pos + V)
    p_neg = (neg + 1) / (total_neg + V)

    log_odds = np.log(p_neg / p_pos)

    rows.append((w, pos, neg, total, log_odds))

word_stats = pd.DataFrame(
    rows,
    columns=["word", "pos_count", "neg_count", "total", "log_odds"]
)

word_stats.sort_values("log_odds", ascending=False).head(10)

In [0]:
%pip install wordcloud

In [0]:
# principais negativas
neg_words = (
    word_stats.sort_values("log_odds", ascending=False)
              .head(100)
)

# principais positivas
pos_words = (
    word_stats.sort_values("log_odds", ascending=True)
              .head(100)
)

In [0]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# dicionários palavra -> peso (uso total ou |log_odds|)
pos_freq = {row.word: row.total for _, row in pos_words.iterrows()}
neg_freq = {row.word: row.total for _, row in neg_words.iterrows()}

In [0]:
wc_pos = WordCloud(
    width=800,
    height=400,
    background_color="white"
).generate_from_frequencies(pos_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc_pos, interpolation="bilinear")
plt.axis("off")
plt.title("Palavras mais associadas a NÃO churn (positivas)")
display(plt.gcf())
plt.close()

In [0]:
wc_neg = WordCloud(
    width=800,
    height=400,
    background_color="white"
).generate_from_frequencies(neg_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc_neg, interpolation="bilinear")
plt.axis("off")
plt.title("Palavras mais associadas a churn (negativas)")
display(plt.gcf())
plt.close()

In [0]:
spark_word_stats = spark.createDataFrame(word_stats)
spark_word_stats.write.mode("overwrite").saveAsTable("workspace.churn.word_polarity")

In [0]:
%pip install scikit-learn wordcloud

In [0]:
# ============================================
# Naive Bayes de palavras (positivo x negativo) + WordCloud
# Baseado na ideia do projeto Spark, mas em Python puro (sem Spark)
# Usa a coluna "CustomerFeedback_clean" e "Churn" de um CSV
#  - Churn = 0  -> feedback positivo (não churn)
#  - Churn = 1  -> feedback negativo (churn)
# ============================================

# Instale as dependências (rode UMA vez no ambiente):
# pip install pandas numpy wordcloud matplotlib

import pandas as pd
import numpy as np
from collections import Counter
from math import log10
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# ------------------------------------------------
# 1. Carregar dados
# ------------------------------------------------
# Ajuste o caminho do arquivo aqui:
CSV_PATH = "history_clean.csv"

df = pd.read_csv(CSV_PATH)

# Garante que as colunas necessárias existem
col_label = "Churn"
col_text  = "CustomerFeedback_clean"

if col_label not in df.columns or col_text not in df.columns:
    raise ValueError(f"O CSV precisa ter as colunas '{col_label}' e '{col_text}'.")

# Remove linhas sem texto
df = df.dropna(subset=[col_text])

# Garante tipos corretos
df[col_label] = df[col_label].astype(int)
df[col_text]  = df[col_text].astype(str)

# ------------------------------------------------
# 2. Construir contagens de palavras por classe (0 = não churn, 1 = churn)
# ------------------------------------------------
pos_counter = Counter()  # não churn (positivo)
neg_counter = Counter()  # churn (negativo)

for _, row in df.iterrows():
    label = row[col_label]
    text  = row[col_text].lower()
    tokens = text.split()
    if label == 0:
        for w in tokens:
            pos_counter[w] += 1
    elif label == 1:
        for w in tokens:
            neg_counter[w] += 1
    # Se tiver outros valores de Churn, são ignorados

# ------------------------------------------------
# 3. Calcular log P(palavra | classe) com Laplace
# ------------------------------------------------
vocab = set(pos_counter.keys()) | set(neg_counter.keys())
V = len(vocab)

total_pos = sum(pos_counter.values())
total_neg = sum(neg_counter.values())

alpha = 1  # Laplace smoothing

# modelo: word -> (log_prob_pos, log_prob_neg)
modelo = {}

for w in vocab:
    c_pos = pos_counter.get(w, 0)
    c_neg = neg_counter.get(w, 0)

    # P(w | classe) com Laplace
    p_pos = (c_pos + alpha) / (total_pos + alpha * V)
    p_neg = (c_neg + alpha) / (total_neg + alpha * V)

    log_p_pos = log10(p_pos)
    log_p_neg = log10(p_neg)

    modelo[w] = (log_p_pos, log_p_neg)

# ------------------------------------------------
# 4. Função classificador (opcional, caso queira classificar textos)
# ------------------------------------------------
def classificar_texto(texto):
    """
    Retorna 0 (não churn / positivo) ou 1 (churn / negativo)
    com base no modelo Naive Bayes de palavras.
    """
    texto = str(texto).lower()
    tokens = texto.split()

    score_pos = 0.0
    score_neg = 0.0

    for p in tokens:
        if p in modelo:
            log_p_pos, log_p_neg = modelo[p]
            score_pos += log_p_pos
            score_neg += log_p_neg

    # Se a soma de log-probs for maior em churn, retorna 1 (negativo)
    return 1 if score_neg > score_pos else 0

# Exemplo de uso:
# print(classificar_texto("service is terrible and very slow"))
# print(classificar_texto("great support, very happy"))

# ------------------------------------------------
# 5. Medir polaridade de cada palavra (positivo x negativo)
#    score = log P(w | churn=1) - log P(w | churn=0)
#    score > 0 -> palavra mais associada a churn (negativa)
#    score < 0 -> palavra mais associada a não churn (positiva)
# ------------------------------------------------
palavras = []
log_pos_list = []
log_neg_list = []
score_list   = []
count_list   = []

for w, (log_p_pos, log_p_neg) in modelo.items():
    score = log_p_neg - log_p_pos  # alinhado com "negativo" = churn
    total_count = pos_counter.get(w, 0) + neg_counter.get(w, 0)
    palavras.append(w)
    log_pos_list.append(log_p_pos)
    log_neg_list.append(log_p_neg)
    score_list.append(score)
    count_list.append(total_count)

word_stats = pd.DataFrame({
    "word": palavras,
    "log_prob_pos": log_pos_list,
    "log_prob_neg": log_neg_list,
    "score": score_list,
    "count": count_list
})

# Opcional: filtrar palavras muito raras
min_count = 5
word_stats = word_stats[word_stats["count"] >= min_count].reset_index(drop=True)

# ------------------------------------------------
# 6. Selecionar top palavras positivas e negativas
# ------------------------------------------------
TOP_N = 100

# Negativas: mais associadas a churn (score alto)
neg_words = word_stats.sort_values("score", ascending=False).head(TOP_N)

# Positivas: mais associadas a não churn (score bem negativo)
pos_words = word_stats.sort_values("score", ascending=True).head(TOP_N)

print("Top 10 palavras NEGATIVAS (churn):")
print(neg_words[["word", "score", "count"]].head(10))
print("\nTop 10 palavras POSITIVAS (não churn):")
print(pos_words[["word", "score", "count"]].head(10))

# ------------------------------------------------
# 7. WordCloud das principais palavras negativas e positivas
#    Usamos |score| como peso (quanto mais forte a polaridade,
#    maior a palavra na nuvem)
# ------------------------------------------------
neg_freq = {row.word: abs(row.score) for _, row in neg_words.iterrows()}
pos_freq = {row.word: abs(row.score) for _, row in pos_words.iterrows()}

# Nuvem de palavras NEGATIVAS (churn) – em VERMELHO
wc_neg = WordCloud(
    width=800,
    height=400,
    background_color="white",
    colormap="Reds"  # <- paleta vermelha
).generate_from_frequencies(neg_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc_neg, interpolation="bilinear")
plt.axis("off")
plt.title("Palavras mais associadas a churn (negativas)")
plt.show()

# Nuvem de palavras POSITIVAS (não churn) – em VERDE
wc_pos = WordCloud(
    width=800,
    height=400,
    background_color="white",
    colormap="Greens"  # <- paleta verde
).generate_from_frequencies(pos_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc_pos, interpolation="bilinear")
plt.axis("off")
plt.title("Palavras mais associadas a NÃO churn (positivas)")
plt.show()

# ------------------------------------------------
# 8. (Opcional) Salvar a tabela de polaridade em CSV
# ------------------------------------------------
# word_stats.to_csv("word_polarity_naive_bayes.csv", index=False)